In [4]:
from __future__ import print_function
from PIL import Image
import cv2
import numpy as np
import math
import string
import sys
import os

In [5]:
part=0
rowNumber= 1
outputRowPath= '/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/wordWise/output/words'


In [6]:
while(True):
    rowDirectory= 'row'+ str(rowNumber)
    finalRowPath= os.path.join(outputRowPath, rowDirectory)
    img=cv2.imread('/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/horizontal/output/ip_11/output/row' +str(part)+'.jpg')
    if img is None:
        break
    
    os.makedirs(finalRowPath,exist_ok = True)
    rows,cols,w = img.shape
    
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, threshold = cv2.threshold(gray_image,200,255,cv2.THRESH_BINARY)
    #dilating
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2000))
    dilation = cv2.dilate(threshold,kernel,iterations = 50)
    th2= dilation.copy()

    ret, threshold = cv2.threshold(th2,200,255,cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2000))
    erosion = cv2.erode(threshold,kernel,iterations = 11)
    
    ret, threshold1 = cv2.threshold(erosion,200,255,cv2.THRESH_BINARY_INV)
    # cv2.imwrite('/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/wordWise/output'+str(part)+'.jpg', threshold1)
    k= threshold1.copy()

    #defining the edges
    edges = cv2.Canny(k,50,150,apertureSize = 3)

    m=[]
    minLineLength = 100
    maxLineGap = 10
    lines = cv2.HoughLinesP(edges,1,np.pi/180,15,minLineLength,maxLineGap)
    for x in range(0, len(lines)):
            for x1,y1,x2,y2 in lines[x]:
                    m.append(((x1,y1),(x2,y2)))

    sorted_m=sorted(m, key=lambda x: x[0][0])

    #drawing lines
    for i in range(len(sorted_m)):
            cv2.line(img,sorted_m[i][0],sorted_m[i][1],(0,0,255),2)
    cv2.imwrite('/Users/abhishekpandey/Desktop/stuff/MajorProject/main/segmentation/wordWise/output/words/Output_1_row_'+str(part)+'.jpg', img)
    hough_lines= img.copy()
    p=[]
    sorted_m.append(((cols,rows),(cols,0)))
    size= (28, 28)
    wordNumber= 1
    count= 0
    for i in range(len(sorted_m)-1):
        p.append(hough_lines[sorted_m[i][1][1]:sorted_m[i][0][1],sorted_m[i][0][0]: sorted_m[i+1][0][0]])
        image = cv2.cvtColor(p[i], cv2.COLOR_BGR2GRAY)
        ret, o = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
        wordDirectory= 'word'+ str(i)+'.jpg'
        wordPath= os.path.join(finalRowPath, wordDirectory)
        if cv2.countNonZero(o) >0:
            cv2.imwrite(wordPath, o)
    part=part+1
    rowNumber= rowNumber+1
print("code complete")

code complete
